In [6]:
!pip install langchain langchain-openai -q
!pip install faiss-cpu -q
!pip install langchain_community -q
!pip install tiktoken -q

In [15]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA

In [16]:
# OpenAI API 키 설정
import os
os.environ["OPENAI_API_KEY"] = ''

# 1. 문서 로드
# 텍스트 파일을 사용해 문서를 로드
loader = TextLoader('documents.txt')
documents = loader.load()

In [17]:
# 2. 문서를 임베딩 및 인덱싱
# 문서를 벡터로 변환 후, FAISS 벡터 저장소에 저장
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(documents, embeddings)

In [18]:
# 3. Fine_tuned GPT 모델을 사용한 LLM 생성
# Fine_tuned된 GPT 모델을 사용할 수 있도록 설정
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.3)

In [19]:
# 4. 문서 검색 개수 조정 (최대 5개의 문서 검색)
# 검색할 문서의 개수를 5개로 설정하여 모든 문서가 context에 반영 될 수 있도록 설정
retriever = vector_store.as_retriever()

In [20]:
# 5. RAG 구현
# RetrievalQA 체인을 사용해 검색된 문서를 기반으로 LLM을 통해 답변을 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,                        # Fine_tuned 된 GPT 모델
    chain_type="stuff",             # 'stuff' 체인을 사용하여 검색된 문서들을 단일 입력으로 결합하여 처리
    retriever=retriever,            # 검색할 문서의 수를 5개로 설정한 retriever
    return_source_documents=True    # 검색된 문서를 포함하여 반환할지 여부를 지정
)

In [38]:
# 6. 질문에 대한 답변생성
query = """
입사하고 1년하고 퇴직했는데 왜 연차 18개 줫을까요?

별로 안쓰고 6개만 쓰고 퇴사 햇는데 남은 12개는 연차수당 많이 받겟네요?
왜 처음 하나씩 주더니 올해보니까 총18개나 되버렷다는...왜 그럴까요?전 딱 1년하고 퇴사햇어요
"""
result = qa_chain.invoke({'query': query})      # invoke 메서드를 사용하여 query 전달

# 7. 결과 출력
print(result["result"])

연차 유급휴가는 근로자가 1년 동안 80% 이상 출근한 경우 15일을 부여받게 됩니다. 하지만, 1년 미만인 근로자에게는 1개월 개근 시 1일의 유급휴가가 주어지며, 1년 이상 근로한 경우에는 추가로 가산휴가가 발생할 수 있습니다. 

따라서, 만약 당신이 1년 동안 근무하면서 80% 이상 출근했다면 기본적으로 15일의 연차가 주어지고, 추가로 계속 근로한 기간에 따라 가산휴가가 발생했을 가능성이 있습니다. 

연차가 총 18개가 된 이유는 다음과 같은 경우일 수 있습니다:
1. 기본 15일에 추가로 가산휴가가 발생했을 경우.
2. 사용하지 않은 연차가 누적되었을 경우.

퇴사 시 남은 연차는 연차수당으로 지급받을 수 있습니다. 연차수당은 사용하지 않은 연차에 대해 통상임금에 해당하는 금액을 지급받는 것이므로, 남은 12개의 연차에 대해 수당을 받을 수 있습니다. 

정확한 사유는 회사의 취업규칙이나 연차 유급휴가 부여 기준에 따라 다를 수 있으므로, 인사팀이나 관련 부서에 문의해보는 것이 좋습니다.


In [24]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain

import os
os.environ["OPENAI_API_KEY"] = ''

# 1. 문서 로드
loader = TextLoader('documents.txt')
documents = loader.load()

# 2. 문서를 임베딩 및 인덱싱
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(documents, embeddings)

# 3. 시스템 메시지 설정
system_message = """
    당신은 근로자들을 상담해주는 근로자 상담 관련 챗봇입니다.
    - 법률에 관한 db는 사용자가 관련된 법이나 법 조항을 물어보고 꼭 필요할때 사용하세요
    - 당신의 역할은 근로기준법을 모르는 사람에게 관련 법과 이에 관한 제도들을 설명해주는 것입니다.
    - 답변이 정확하지 않을 가능성이 높을 경우 잘 모른다고 답변해야 됩니다. 부정확한 답변은 하지 않는게 좋습니다.
    - 질문이 구체적이라면 그 방법도 구체적으로 알려주는것이 좋습니다.
    - 유저의 질문에 정보가 많이 없을때는 한번 더 질문을 해서 더 구체적인 답변을 주는것이 좋습니다. 예를 들면 월급이나 수당관련되서는 날짜를 다시 물어본 후 계산을 해줄지 한번 더 물어본다음 계산을 해주면 원하는 답변을 해줄 수 있어요.
    - 또한 유저에게 중요해 보이는 부분에 대해서 다시 2~3개의 질문으로 유저의 질문을 도와주는 것도 중요합니다.
    - 답변에 대해서는 최대한 상세하게 알려주세요.
    - 금액 산정에 관한 질문에는 돈의 성격에 따라서 세금이나, 수당, 공제 등 여러가지 변수들이 있을 수 있음으로 성격에 맞게 이를 명시하여 주세요.
"""

# 4. 프롬프트 템플릿 설정
prompt_template = PromptTemplate(
    template=system_message + "\n\n질문: {query}",
    input_variables=["query"]
)

# 5. Fine-tuned GPT 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.3)

# 6. 문서 검색 개수 조정
retriever = vector_store.as_retriever()

# 7. 문서 검색 및 답변 생성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# 8. 프롬프트 체인 생성
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

# 9. 질문에 대한 답변 생성
query = """
입사하고 1년하고 퇴직했는데 왜 연차 18개 줫을까요?

별로 안쓰고 6개만 쓰고 퇴사 햇는데 남은 12개는 연차수당 많이 받겟네요?
왜 처음 하나씩 주더니 올해보니까 총18개나 되버렷다는...왜 그럴까요?전 딱 1년하고 퇴사햇어요
"""

# RAG를 통한 문서 검색
result = qa_chain.invoke({'query': query})

# 프롬프트 체인을 통한 상세 답변 생성
prompt_result = llm_chain.run({"query": query})

# 10. 결과 출력
print("RAG Result:")
print(result["result"])

print("\nPrompt Template Result:")
print(prompt_result)


c:\Users\Administrator\anaconda3\envs\myenv\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


RAG Result:
연차 유급휴가는 근로자가 1년간 80% 이상 출근한 경우 15일을 부여받고, 1년 이상 근로한 경우에는 추가로 가산휴가가 주어질 수 있습니다. 

귀하의 경우, 1년 동안 근로하셨고, 1년 이상 근무한 근로자에게는 15일의 기본 연차 외에 추가로 3일의 가산휴가가 주어질 수 있습니다. 따라서 총 18일의 연차가 부여된 것으로 보입니다.

퇴사 시 사용하지 않은 연차에 대해서는 연차수당을 받을 수 있으므로, 남은 12개의 연차에 대한 수당을 받을 수 있을 것입니다. 연차는 사용하지 않으면 소멸되지 않고, 퇴사 시 수당으로 지급되기 때문입니다. 

이러한 규정은 근로기준법에 따라 정해져 있으므로, 연차에 대한 정확한 내용은 근로계약서나 회사의 취업규칙을 참고하시기 바랍니다.

Prompt Template Result:
연차 유급휴가는 근로기준법에 따라 근로자가 1년 동안 근무한 경우 최소 15일의 유급휴가를 받을 수 있습니다. 하지만, 근로자가 1년 이상 근무하게 되면 추가적인 연차가 발생할 수 있습니다. 

1. **연차 발생 원리**: 
   - 근로기준법에 따르면, 1년 동안 근무한 근로자는 15일의 연차를 부여받습니다. 이후 1년을 추가로 근무할 때마다 1일의 연차가 추가로 발생합니다. 
   - 예를 들어, 1년 근무 후 퇴직하는 경우, 기본 15일의 연차가 발생하고, 만약 추가로 1년을 더 근무하지 않았다면 추가 연차는 발생하지 않습니다.

2. **연차 수당**: 
   - 퇴사 시 사용하지 않은 연차에 대해서는 연차 수당으로 지급받을 수 있습니다. 따라서, 남은 12개의 연차에 대해서는 퇴사 시 수당을 받을 수 있습니다.

3. **연차 수당 계산**: 
   - 연차 수당은 사용하지 않은 연차 일수에 근로자의 1일 평균 임금을 곱하여 계산됩니다. 평균 임금은 기본급, 수당 등을 포함하여 계산됩니다.

4. **왜 18개가 되었는가?**: 
   - 연차가 18개로 보이는 이유는, 근로계약서나 회사의 내부 규정에 따라 추가적인 

중요한건 답변이 건강해지고 있다는점...

In [39]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
import os

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = ''

# 1. 문서 로드
loader = TextLoader('documents.txt')
documents = loader.load()

# 2. 문서를 임베딩 및 인덱싱
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(documents, embeddings)

# 3. 문서 검색
retriever = vector_store.as_retriever()

# 4. 프롬프트 템플릿 정의
template = """
당신은 근로자들을 상담해주는 근로자 상담 관련 챗봇입니다.
    - 법률에 관한 db는 사용자가 관련된 법이나 법 조항을 물어보고 꼭 필요할때 사용하세요
    - 당신의 역할은 근로기준법을 모르는 사람에게 관련 법과 이에 관한 제도들을 설명해주는 것입니다.
    - 답변이 정확하지 않을 가능성이 높을 경우 잘 모른다고 답변해야 됩니다. 부정확한 답변은 하지 않는게 좋습니다.
    - 질문이 구체적이라면 그 방법도 구체적으로 알려주는것이 좋습니다.
    - 유저의 질문에 정보가 많이 없을때는 한번 더 질문을 해서 더 구체적인 답변을 주는것이 좋습니다. 예를 들면 월급이나 수당관련되서는 날짜를 다시 물어본 후 계산을 해줄지 한번 더 물어본다음 계산을 해주면 원하는 답변을 해줄 수 있어요.
    - 한 가지 주제에 대한 질문에 답변 후, 사용자가 놓쳤을 수 있는 관련 주제에 대해 추가 질문을 제안해 주세요.
    - 답변에 대해서는 최대한 상세하게 알려주세요.
    - 금액 산정에 관한 질문에는 돈의 성격에 따라서 세금이나, 수당, 공제 등 여러가지 변수들이 있을 수 있음으로 성격에 맞게 이를 명시하여 주세요.
    - 법률적 조언과 일반적인 정보 제공의 차이를 명확히 해주세요. 법률에 관한 정보를 줄 수는 있으나, 전문 변호사나 상담 기관에 문의할 것을 권장해 주세요.
문서 내용: {context}
질문: {query}
"""
prompt_template = PromptTemplate(template=template, input_variables=["context", "query"])

# 5. LLM 체인 설정
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.3)
llm_chain = LLMChain(
    prompt=prompt_template,
    llm=llm
)

# 6. RAG 구현
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# 7. 질문에 대한 답변 생성
query = input("질문을 입력해주세요!")
# 문서 검색
docs = retriever.get_relevant_documents(query)
# LLM 체인으로 프롬프트 템플릿에 문서 내용과 질문을 입력
context = " ".join([doc.page_content for doc in docs])
result = llm_chain.run({"context": context, "query": query})

# 8. 결과 출력
print("Result:", result)

Result: 네, 근로기준법에서는 휴가에 대한 규정을 명시하고 있습니다. 주요 내용은 다음과 같습니다:

1. **연차 유급휴가 (제60조)**:
   - 사용자는 1년간 80% 이상 출근한 근로자에게 15일의 유급휴가를 제공해야 합니다.
   - 1년 미만 근로자에게는 1개월 개근 시 1일의 유급휴가를 주어야 합니다.
   - 3년 이상 계속 근로한 근로자에게는 최초 1년을 초과하는 계속 근로 연수 매 2년에 대해 1일을 가산한 유급휴가를 제공해야 하며, 총 휴가 일수는 25일을 한도로 합니다.
   - 사용자는 근로자가 청구한 시기에 휴가를 주어야 하며, 그 기간에 대해 통상임금 또는 평균임금을 지급해야 합니다.

2. **생리휴가 (제73조)**:
   - 여성 근로자가 청구하면 월 1일의 생리휴가를 제공해야 합니다.

3. **출산전후휴가 (제74조)**:
   - 사용자는 임신 중인 여성에게 출산 전후로 총 90일(한 번에 둘 이상 자녀를 임신한 경우 120일)의 휴가를 제공해야 하며, 이 중 최초 60일은 유급으로 지급해야 합니다.

이 외에도 근로기준법에는 다양한 휴가 관련 규정이 포함되어 있습니다. 더 구체적인 내용이나 다른 질문이 있으시면 말씀해 주세요!


In [49]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import os

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = (
    ""
)

# 1. 문서 로드
loader = TextLoader("documents.txt", encoding="utf-8")
documents = loader.load()

# 2. 문서를 임베딩 및 인덱싱
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(documents, embeddings)

# 3. 문서 검색을 위한 retriever 생성
retriever = vector_store.as_retriever()

# 4. 프롬프트 템플릿 정의
template = """
## 당신은 근로자들을 상담해주는 근로자 상담 관련 챗봇입니다.
- 법률에 관한 db는 사용자가 관련된 법이나 법 조항을 물어보고 꼭 필요할때 사용하세요.
- 당신의 역할은 근로기준법을 모르는 사람에게 관련 법과 이에 관한 제도들을 설명해주는 것입니다.
- 답변이 정확하지 않을 가능성이 높을 경우 잘 모른다고 답변해야 됩니다. 부정확한 답변은 하지 않는게 좋습니다.
- 질문이 구체적이라면 그 방법도 구체적으로 알려주는것이 좋습니다.
- 유저의 질문에 정보가 많이 없을때는 한번 더 질문을 해서 더 구체적인 답변을 주는것이 좋습니다. 예를 들면 월급이나 수당관련되서는 날짜를 다시 물어본 후 계산을 해줄지 한번 더 물어본다음 계산을 해주면 원하는 답변을 해줄 수 있어요.
- 한 가지 주제에 대한 질문에 답변 후, 사용자가 놓쳤을 수 있는 관련 주제에 대해 추가 질문을 제안해 주세요.
- 답변에 대해서는 최대한 상세하게 알려주세요.
- 금액 산정에 관한 질문에는 돈의 성격에 따라서 세금이나, 수당, 공제 등 여러가지 변수들이 있을 수 있음으로 성격에 맞게 이를 명시하여 주세요.
- 법률적 조언과 일반적인 정보 제공의 차이를 명확히 해주세요. 법률에 관한 정보를 줄 수는 있으나, 전문 변호사나 상담 기관에 문의할 것을 권장해 주세요.

1. 

문서 내용: {context}
질문: {human_input}

응답:
"""
prompt_template = PromptTemplate(
    template=template, input_variables=["context", "human_input"]
)

# 5. LLM 생성
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0.3)

# 6. 메모리 생성
memory = ConversationBufferMemory(input_key="human_input", output_key="output")

# 7. 체인 생성
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    memory=memory,
    output_key="output"
)

# 8. 대화 처리 함수
def handle_query(query):
    # 문서에서 관련된 내용 검색
    documents = retriever.get_relevant_documents(query)
    context = "\n".join([doc.page_content for doc in documents])
    
    # LLM으로 응답 생성
    result = llm_chain({"context": context, "human_input": query})
    return result["output"]

# 9. 사용자로부터 질문을 입력받아 응답 생성
while True:
    query = input("질문을 입력해주세요 (종료하려면 'exit' 입력): ")
    if query.lower() == 'exit':
        break
    response = handle_query(query)
    print("응답:", response)

# 대화 종료 후 메모리 데이터를 JSON 파일로 저장
import json

# 메모리 데이터를 JSON 직렬화 가능한 형식으로 변환
memory_data = [
    {
        "type": message.type,
        "content": message.content
    }
    for message in memory.chat_memory.messages
]

응답: 퇴직금과 연차수당이 미지급된 경우, 노동청에 신고하는 절차는 다음과 같습니다:

1. **신고서 작성**: 노동청에 제출할 신고서를 작성해야 합니다. 퇴직금과 연차수당 미지급에 대한 내용을 포함해야 합니다. 이때, 미지급 사실을 증명할 수 있는 서류(예: 근로계약서, 급여명세서 등)를 함께 첨부하는 것이 좋습니다.

2. **신고서 제출**: 작성한 신고서를 관할 지방고용노동관서에 제출합니다. 직접 방문하거나 우편으로 제출할 수 있습니다. 또한, 일부 지역에서는 온라인으로 신고할 수 있는 시스템이 마련되어 있으니 확인해 보세요.

3. **증빙자료 준비**: 신고서와 함께 제출할 증빙자료를 준비합니다. 예를 들어, 퇴직금과 연차수당이 미지급된 기간, 금액, 관련 서류 등을 포함해야 합니다.

4. **신고 후 절차**: 신고 후, 노동청에서는 해당 사건에 대해 조사 및 심사를 진행합니다. 필요시 추가 자료를 요청할 수 있으며, 조사 결과에 따라 사용자가 미지급된 금액을 지급하도록 명령할 수 있습니다.

5. **결과 통지**: 조사 결과는 신고자에게 통지됩니다. 만약 사용자가 미지급된 금액을 지급하지 않을 경우, 추가적인 법적 조치를 고려할 수 있습니다.

신고하는 과정에서 도움이 필요하거나 법적 조언이 필요하다면, 전문 변호사나 노무사에게 상담을 받는 것도 좋은 방법입니다.

혹시 퇴직금이나 연차수당의 구체적인 금액이나 기간에 대한 정보가 필요하신가요? 추가적인 질문이 있으시면 말씀해 주세요!
응답: 근로조건 위반에 대한 손해배상을 청구하려면, 다음과 같은 서류를 준비해야 합니다:

1. **근로계약서 사본**: 근로자가 사용자와 체결한 근로계약서의 사본이 필요합니다.
2. **사용자가 근로조건을 위반하였다는 사실을 증명하는 자료**: 이는 급여대장, 근무기관 확인 자료 외에도 사용자가 근로조건을 위반한 사실을 입증할 수 있는 모든 관련 문서가 포함됩니다. 예를 들어, 급여의 미지급, 근로시간의 위반, 휴가 미부여 등의 증거가 될 수 있는 자료가 필요

In [50]:
# 메모리 데이터를 JSON 파일로 저장
with open('memory_data.json', 'w', encoding='utf-8') as f:
    json.dump(memory_data, f, ensure_ascii=False, indent=2)

print("메모리 데이터가 'memory_data.json' 파일에 저장되었습니다.")

메모리 데이터가 'memory_data.json' 파일에 저장되었습니다.
